# <span style="color:#F72585">Optimización con torch</span>


## <span style="color:#4361EE">Introducción</span>

Ahora que tenemos un modelo y datos, es hora de entrenar, validar y probar nuestro modelo optimizando sus parámetros en nuestros datos. 

Entrenar un modelo es un proceso iterativo; en cada iteración (llamada época) el modelo hace una suposición sobre la salida, calcula el error en su suposición (pérdida), recopila las derivadas del error con respecto a sus parámetros (como vimos en la sección anterior) y optimiza estos parámetros usando el descenso de gradiente.

## <span style="color:#4361EE">Implementación completa</span>

In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

# datos
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# modelo
device = 'cuda' if torch.cuda.is_available() else 'cpu'
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)


# hiperparámetros
learning_rate = 1e-3
batch_size = 64
epochs = 5

# loop de entrenamiento

# función de pérdida
loss_fn = nn.CrossEntropyLoss()

# Optimizador
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# función de entrenamiento
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # calcula predicción y pérdida
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # backpropagation
        optimizer.zero_grad()# por defecto el calculo del gradiente es acumulativo
        loss.backward() # clacula y acumula los gradientes
        optimizer.step()# paso de optimización x += grad
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

# función de validación
def test_loop(testloader, model, loss_fn):
    size = len(testloader.dataset)
    num_batches = len(testloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in testloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# ciclo de entrenamiento
epochs = 10
for t in range(epochs):
    print(f'Epoch {t+1}\n -------------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!')


Epoch 1
 -------------------------
loss: 2.305429 [    0/60000]
loss: 2.303908 [ 6400/60000]
loss: 2.300803 [12800/60000]
loss: 2.296808 [19200/60000]
loss: 2.298529 [25600/60000]
loss: 2.289145 [32000/60000]
loss: 2.283352 [38400/60000]
loss: 2.281261 [44800/60000]
loss: 2.288461 [51200/60000]
loss: 2.278966 [57600/60000]
Test Error: 
 Accuracy: 28.2%, Avg loss: 2.269166 

Epoch 2
 -------------------------
loss: 2.282544 [    0/60000]
loss: 2.280299 [ 6400/60000]
loss: 2.252230 [12800/60000]
loss: 2.242313 [19200/60000]
loss: 2.263425 [25600/60000]
loss: 2.245561 [32000/60000]
loss: 2.236371 [38400/60000]
loss: 2.221991 [44800/60000]
loss: 2.227698 [51200/60000]
loss: 2.239628 [57600/60000]
Test Error: 
 Accuracy: 29.2%, Avg loss: 2.204753 

Epoch 3
 -------------------------
loss: 2.223277 [    0/60000]
loss: 2.226054 [ 6400/60000]
loss: 2.164122 [12800/60000]
loss: 2.155463 [19200/60000]
loss: 2.211769 [25600/60000]
loss: 2.180077 [32000/60000]
loss: 2.165720 [38400/60000]
loss: 2.

## <span style="color:#4361EE">Referencias</span> 

1. Basado en los [tutoriales de Pytorch](https://pytorch.org/tutorials/)
1. [Deep learning for coders with FastAI and Pytorch](http://library.lol/main/F13E85845AE48D9FD7488FE7630A9FD3)